In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer 
from transformers import BertForSequenceClassification
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv(r"D:\epita class notes\semester - 3\action learnign\project repository\Hate_speech_detection_using_data_augmentation\Hate_speech_detection_using_data_augmentation\data\cleaned_dataset\labeled_data_cleaned.csv")
df

,count,hate_speech,offensive_language,neither,corrected_tweet,class
0,3,0,0,3,as a woman you shouldn't complain about cleani...,2
1,3,0,3,0,boy date coldtyga own bad for coffin dat hoe i...,1
2,3,0,3,0,dawn you ever fuck a bitch and she start to cr...,1
3,3,0,2,1,she look like a granny,1
4,6,0,6,0,they shit you hear about me might be true or i...,1
...,...,...,...,...,...,...
995,3,0,3,0,munda out here sucking bitches howdhow,1
996,3,0,3,0,bitch if your they hobbit you need to let me k...,1
997,3,0,2,1,these folks so bad in just here to talk trash,1
998,6,0,6,0,brittany bitch a my dog man,1


In [6]:
texts = df['corrected_tweet']
label = df['class']
texts, label

(0      as a woman you shouldn't complain about cleani...
 1      boy date coldtyga own bad for coffin dat hoe i...
 2      dawn you ever fuck a bitch and she start to cr...
 3                                 she look like a granny
 4      they shit you hear about me might be true or i...
                              ...                        
 995               munda out here sucking bitches howdhow
 996    bitch if your they hobbit you need to let me k...
 997        these folks so bad in just here to talk trash
 998                          brittany bitch a my dog man
 999                            sch bitch selling her ass
 Name: corrected_tweet, Length: 1000, dtype: object,
 0      2
 1      1
 2      1
 3      1
 4      1
       ..
 995    1
 996    1
 997    1
 998    1
 999    1
 Name: class, Length: 1000, dtype: int64)

In [7]:
train_texts, val_texts, train_label, val_label = train_test_split(texts, label, test_size=0.2, random_state = 42)


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer

D:\miniconda\miniconda\envs\action_learning\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [9]:
def tokenize_data(texts, labels):
    inputs = tokenizer(list(texts), truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs['labels'] = torch.tensor(labels.values)
    return inputs

In [19]:
train_inputs = tokenize_data(train_texts, train_label)
val_inputs = tokenize_data(val_texts, val_label)

In [20]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

D:\miniconda\miniconda\envs\action_learning\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
device = torch.device("cpu")
device

device(type='cpu')

In [22]:
model = model.to(device)
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
optimizer = AdamW(model.parameters(), lr=0.01)
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    lr: 0.01
    weight_decay: 0.0
)

In [25]:
def train_model(model, train_inputs, epochs=3):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in train_inputs.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [26]:
def evaluate_model(model, val_inputs):
    model.eval()
    with torch.no_grad():
        inputs = {key: val.to(device) for key, val in val_inputs.items()}
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        accuracy = accuracy_score(val_inputs['labels'].cpu().numpy(), preds)
        print(f"Validation Accuracy: {accuracy}")

In [27]:
train_model(model, train_inputs)

Epoch 1/3, Loss: 1.2239773273468018
Epoch 2/3, Loss: 1.9949935674667358
Epoch 3/3, Loss: 13.615461349487305
